In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
from config import g_key

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.dropna(inplace = True)


weather_df.dtypes

Unnamed: 0            int64
Country              object
City                 object
Date                  int64
Latitude            float64
Longitude           float64
Temperature (F)     float64
Humidity (%)          int64
Cloudiness (%)        int64
Wind Speed (mph)    float64
dtype: object

In [5]:
cities = weather_df[["Latitude", "Longitude"]]

humidity = weather_df["Humidity (%)"]

In [6]:
fig = gmaps.figure(map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
hotel_df = weather_df.loc[(weather_df["Temperature (F)"] > 65) & (weather_df["Temperature (F)"] < 80) & (weather_df["Cloudiness (%)"] < 25) & (weather_df["Wind Speed (mph)"] < 10)].copy()
hotel_df["Hotel Name"] = ""

hotel_df

,Unnamed: 0,Country,City,Date,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
11,11,AU,Swan Hill,1605340005,-35.34,143.55,77.07,28,0,-35.34,
13,13,PF,Atuona,1605340005,-9.80,-139.03,77.70,76,24,-9.80,
34,34,TO,Vaini,1605340008,-21.20,-175.20,77.00,88,20,-21.20,
46,46,ZA,East London,1605340009,-33.02,27.91,75.20,44,0,-33.02,
73,73,NG,Ilorin,1605340013,8.50,4.55,71.60,73,0,8.50,
79,79,TL,Airai,1605340013,-8.93,125.41,75.45,61,13,-8.93,
80,80,BR,Itarema,1605340014,-2.92,-39.92,79.23,79,0,-2.92,
88,88,NC,Vao,1605340015,-22.67,167.48,75.20,84,0,-22.67,
99,99,BR,Cidreira,1605340016,-30.18,-50.21,68.45,92,19,-30.18,
156,156,AU,Port Lincoln,1605339794,-34.73,135.87,66.36,60,0,-34.73,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "Hotels",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


KeyboardInterrupt: 

In [ ]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
 # Add marker layer ontop of heat map


# Display figure